In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [20]:
driver = webdriver.Chrome()
# Fazer login
driver.get('https://sellit-app-web-prod.azurewebsites.net/')

In [3]:
# Feito o login
driver.get('https://sellit-app-web-prod.azurewebsites.net/leads')

In [24]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- PRÉ-REQUISITO: A variável 'driver' já deve existir no seu Notebook ---

# Lista para armazenar os resultados para o relatório final
resultados_leads = []

# >>>>>>>>>>>> SELEÇÃO DE LEADS <<<<<<<<<<<<<<<
try:
    limite_input = input("Quantos leads você quer processar? (Deixe em branco e pressione Enter para processar todos): ")
    if limite_input == '':
        LIMITE_DE_LEADS = 9999
        print("Ok, processando todos os leads encontrados.")
    else:
        LIMITE_DE_LEADS = int(limite_input)
except ValueError:
    print("Entrada inválida. Usando o limite padrão de 2000.")
    LIMITE_DE_LEADS = 2000
# ---------------------------------------------

wait = WebDriverWait(driver, 10)
print(f"--- Iniciando a automação (limite de {LIMITE_DE_LEADS} leads) ---")

leads_processados_total = 0

# --- LOOP PRINCIPAL COM PAGINAÇÃO ---
while leads_processados_total < LIMITE_DE_LEADS:
    try:
        print(f"\n--- Verificando página: {driver.current_url} ---")
        # Espera que a tabela esteja presente e visível
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'table thead ~ thead')))
        linhas = driver.find_elements(By.CSS_SELECTOR, 'table thead ~ thead')
        print(f"Sucesso! {len(linhas)} leads encontrados nesta página.")

        for i in range(len(linhas)):
            if leads_processados_total >= LIMITE_DE_LEADS:
                print(f"\nLimite de {LIMITE_DE_LEADS} leads atingido. Interrompendo o processo.")
                break

            linha_atual = driver.find_elements(By.CSS_SELECTOR, 'table thead ~ thead')[i]
            nome_lead = f"Lead na posição {i+1}"

            try:
                # --- CORREÇÃO DE FOCO E TIMING ---
                # 1. Rola a linha para o centro da tela
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", linha_atual)
                # 2. Rola um pouco mais para baixo (150 pixels) para garantir que o cabeçalho não atrapalhe
                driver.execute_script("window.scrollBy(0, -150);")
                # 3. Pausa explícita de 1 segundo para a interface gráfica assentar.

                nome_lead = linha_atual.find_element(By.CSS_SELECTOR, 'a.sc-ldtZsI').text
                expira = linha_atual.find_element(By.CSS_SELECTOR, 'td:nth-child(13)').text

                print(f"\nAnalisando Lead: {nome_lead} - Status: {expira}")
                expirado = expira.split()
                expirado_int = int(expirado[0])

                if expirado_int > 30:
                    print(f'O lead {nome_lead} tem mais de 30 dias para expirar (pulado)')
                    resultados_leads.append({'Nome do Lead': nome_lead, 'Status': {expirado_int}})
                    continue

                print(f"--> ATUALIZANDO lead '{nome_lead}'...")

                # Usando o clique normal que você confia, agora que a visão está limpa
                botao_acao = wait.until(EC.element_to_be_clickable(linha_atual.find_element(By.CSS_SELECTOR, 'a[aria-label="Criar tarefa"]')))
                botao_acao.click()

                # O restante do seu código que já funciona...
                wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.MuiDialog-root.sc-imWYAI.dMYvXE')))
                
                seletor_seta_tarefa = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(1) > div > div > div > div > div > button.MuiButtonBase-root.MuiIconButton-root.MuiAutocomplete-popupIndicator'
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_seta_tarefa))).click()
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li#combo-box-demo-option-2'))).click()
                
                texto_descricao = f'Ofertei, Home Ipiranga, United, Addres, Acuruí, Home boutique, Palazzo, Duet, Edition, High line, Residences, Sence e lançamento na Barão do Triunfo e outro na vila mascote. Quer ser avisado de todos os lançamentos! Totalmente aberto as localizações.'
                seletor_descricao = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(5) > div > div > div > textarea:nth-child(1)'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_descricao))).send_keys(texto_descricao)
                seletor_checkbox_xpath = '/html/body/div[5]/div[3]/div/div[2]/div/h2/div[1]/div[6]/div[2]/label/span[1]/span[1]/input'
                checkbox = wait.until(EC.presence_of_element_located((By.XPATH, seletor_checkbox_xpath)))
                checkbox.click()
                seletor_seta_resultado = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(7) > div > div > div > div > div > button.MuiButtonBase-root.MuiIconButton-root.MuiAutocomplete-popupIndicator'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_seta_resultado))).click()
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li#combo-box-demo-option-0'))).click()
                texto_resultado = 'Lead contatado, em acompanhamento.'
                seletor_texto_resultado = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(8) > div > div > div > textarea:nth-child(1)'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_texto_resultado))).send_keys(texto_resultado)
                seletor_salvar = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-fHjqPf.jyGuqJ > button.MuiButtonBase-root.MuiButton-root.MuiButton-text.sc-feUZmu.dwrURg'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_salvar))).click()
                seletor_fechar = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-fHjqPf.jyGuqJ > button.MuiButtonBase-root.MuiButton-root.MuiButton-outlined.sc-dAbbOL.dIjAJu'
                time.sleep(1)
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_fechar))).click()
                

                print(f"--> Lead '{nome_lead}' atualizado com SUCESSO.")
                resultados_leads.append({'Nome do Lead': nome_lead, 'Status': 'Atualizado com Sucesso'})
                leads_processados_total += 1

            except Exception as e:
                print(f"!!!!!!!! ERRO ao processar o lead '{nome_lead}': {e}")
                resultados_leads.append({'Nome do Lead': nome_lead, 'Status': f'ERRO - {str(e)[:100]}...'})
                break

        if leads_processados_total >= LIMITE_DE_LEADS:
            break

        # --- LÓGICA DE PAGINAÇÃO ---
        try:
            driver.refresh()
            time.sleep(4)
        except NoSuchElementException:
            print("\nNão foi possível encontrar o botão 'próximo'. Fim de todos os leads.")
            break

    except TimeoutException:
        print('Passou mais de 2 minutos não achou')
        print("\nERRO CRÍTICO: Tabela de leads não foi encontrada na página. O loop de paginação vai parar.")
        break

# --- FIM DE TODOS OS LOOPS ---

# --- 3. CRIAR O RELATÓRIO CSV ---
try:
    print("\n\n--- Criando relatório final ---")
    nome_arquivo_csv = 'relatorio_leads.csv'
    headers = ['Nome do Lead', 'Status']

    with open(nome_arquivo_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(resultados_leads)
    
    print(f"Relatório '{nome_arquivo_csv}' criado com sucesso!")

except Exception as e:
    print(f"!!!!!!!! ERRO ao criar o arquivo CSV: {e}")


print("\n\nFim da execução. O navegador continua aberto.")

--- Iniciando a automação (limite de 807 leads) ---


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=139.0.7258.128)
Stacktrace:
	GetHandleVerifier [0x0x7ff7a8576b25+79621]
	GetHandleVerifier [0x0x7ff7a8576b80+79712]
	(No symbol) [0x0x7ff7a830c0ea]
	(No symbol) [0x0x7ff7a82f8355]
	(No symbol) [0x0x7ff7a831cefa]
	(No symbol) [0x0x7ff7a8392845]
	(No symbol) [0x0x7ff7a83b2d02]
	(No symbol) [0x0x7ff7a838af13]
	(No symbol) [0x0x7ff7a8354151]
	(No symbol) [0x0x7ff7a8354ee3]
	GetHandleVerifier [0x0x7ff7a883683d+2962461]
	GetHandleVerifier [0x0x7ff7a8830b5d+2938685]
	GetHandleVerifier [0x0x7ff7a884f71d+3064573]
	GetHandleVerifier [0x0x7ff7a8590c6e+186446]
	GetHandleVerifier [0x0x7ff7a8598a3f+218655]
	GetHandleVerifier [0x0x7ff7a857f914+115956]
	GetHandleVerifier [0x0x7ff7a857fac9+116393]
	GetHandleVerifier [0x0x7ff7a8565ef8+10968]
	BaseThreadInitThunk [0x0x7ff87285e8d7+23]
	RtlUserThreadStart [0x0x7ff873afc34c+44]

In [14]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- PRÉ-REQUISITO: A variável 'driver' já deve existir no seu Notebook ---

# Lista para armazenar os resultados para o relatório final
resultados_leads = []

# >>>>>>>>>>>> SELEÇÃO DE LEADS <<<<<<<<<<<<<<<
try:
    limite_input = input("Quantos leads você quer processar? (Deixe em branco e pressione Enter para processar todos): ")
    if limite_input == '':
        LIMITE_DE_LEADS = 9999
        print("Ok, processando todos os leads encontrados.")
    else:
        LIMITE_DE_LEADS = int(limite_input)
except ValueError:
    print("Entrada inválida. Usando o limite padrão de 2000.")
    LIMITE_DE_LEADS = 2000
# ---------------------------------------------

wait = WebDriverWait(driver, 120)
print(f"--- Iniciando a automação (limite de {LIMITE_DE_LEADS} leads) ---")

leads_processados_total = 0
refresh_count = 0

# --- LOOP PRINCIPAL COM REFRESH ---
while leads_processados_total < LIMITE_DE_LEADS:
    refresh_count += 1
    print(f"\n--- REFRESH #{refresh_count} ---")
    print(f"URL atual: {driver.current_url}")
    
    try:
        # Espera que a tabela esteja presente e visível
        print("Aguardando carregamento da tabela de leads...")
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'table thead ~ thead')))
        linhas = driver.find_elements(By.CSS_SELECTOR, 'table thead ~ thead')
        print(f"Sucesso! {len(linhas)} leads encontrados nesta página.")

        for i in range(len(linhas)):
            if leads_processados_total >= LIMITE_DE_LEADS:
                print(f"\nLimite de {LIMITE_DE_LEADS} leads atingido. Interrompendo o processo.")
                break

            try:
                linha_atual = driver.find_elements(By.CSS_SELECTOR, 'table thead ~ thead')[i]
                nome_lead = f"Lead na posição {i+1}"
                
                # --- CORREÇÃO DE FOCO E TIMING ---
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", linha_atual)
                driver.execute_script("window.scrollBy(0, -150);")
                time.sleep(1)

                # Coleta informações do lead
                try:
                    nome_lead = linha_atual.find_element(By.CSS_SELECTOR, 'a.sc-ldtZsI').text
                except Exception as e:
                    print(f"Erro ao obter nome do lead: {str(e)}")
                    nome_lead = f"Lead {i+1} (nome não encontrado)"
                
                try:
                    expira = linha_atual.find_element(By.CSS_SELECTOR, 'td:nth-child(13)').text
                except Exception as e:
                    print(f"Erro ao obter data de expiração: {str(e)}")
                    resultados_leads.append({'Nome do Lead': nome_lead, 'Status': 'ERRO - Data expiração'})
                    continue

                print(f"\nAnalisando Lead: {nome_lead} - Status: {expira}")
                
                try:
                    expirado_int = int(expira.split()[0])
                except Exception as e:
                    print(f"Erro ao converter dias: {str(e)}")
                    resultados_leads.append({'Nome do Lead': nome_lead, 'Status': 'ERRO - Formato inválido'})
                    continue

                if expirado_int > 30:
                    print(f'O lead {nome_lead} tem mais de 30 dias para expirar (pulado)')
                    resultados_leads.append({'Nome do Lead': nome_lead, 'Status': f'{expirado_int} dias'})
                    continue

                print(f"--> ATUALIZANDO lead '{nome_lead}'...")

                # --- PROCESSO DE ATUALIZAÇÃO ---
                try:
                    print("Clicando no botão de ação...")
                    botao_acao = wait.until(EC.element_to_be_clickable(
                        (linha_atual.find_element(By.CSS_SELECTOR, 'a[aria-label="Criar tarefa"]'))
                    )
                    botao_acao.click()
                except Exception as e:
                    print(f"ERRO NO BOTÃO DE AÇÃO: {str(e)}")
                    resultados_leads.append({'Nome do Lead': nome_lead, 'Status': f'ERRO Botão Ação - {str(e)[:70]}'})
                    continue

                try:
                    print("Aguardando modal...")
                    wait.until(EC.visibility_of_element_located(
                        (By.CSS_SELECTOR, 'div.MuiDialog-root.sc-imWYAI.dMYvXE'))
                except TimeoutException:
                    print("ERRO: Modal não apareceu após 120 segundos")
                    resultados_leads.append({'Nome do Lead': nome_lead, 'Status': 'ERRO Modal não carregou'})
                    continue

                # ... (mantido o restante do código de preenchimento do modal) ...

                print(f"--> Lead '{nome_lead}' atualizado com SUCESSO.")
                resultados_leads.append({'Nome do Lead': nome_lead, 'Status': 'Atualizado com Sucesso'})
                leads_processados_total += 1

            except Exception as e:
                print(f"!!!!!!!! ERRO AO PROCESSAR LEAD: {str(e)}")
                resultados_leads.append({'Nome do Lead': nome_lead, 'Status': f'ERRO Genérico - {str(e)[:100]}'})
                continue

    except TimeoutException:
        print('ERRO CRÍTICO: Tabela de leads não carregou em 120 segundos')
    except Exception as e:
        print(f'ERRO INESPERADO: {str(e)}')

    # --- REFRESH APÓS PROCESSAR PÁGINA ---
    print("\nAtualizando página para buscar novos leads...")
    driver.refresh()
    time.sleep(5)  # Espera para a página recarregar

# --- FIM DO LOOP PRINCIPAL ---

# --- CRIAR RELATÓRIO CSV ---
try:
    print("\n\n--- Criando relatório final ---")
    nome_arquivo_csv = 'relatorio_leads.csv'
    headers = ['Nome do Lead', 'Status']

    with open(nome_arquivo_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(resultados_leads)
    
    print(f"Relatório '{nome_arquivo_csv}' criado com sucesso!")
    print(f"Total de leads processados: {leads_processados_total}/{LIMITE_DE_LEADS}")
    print(f"Total de erros: {len([r for r in resultados_leads if 'ERRO' in r['Status']])}")

except Exception as e:
    print(f"!!!!!!!! ERRO AO CRIAR CSV: {e}")

print("\n\nFim da execução. O navegador continua aberto.")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1876707130.py, line 93)